# JBang Jupyter Kernel Examples

JBang kernel supports using jbang for loading dependencies/resources.

In [ ]:
%jbang properties@jbangdev

In [ ]:
//DEPS com.github.javafaker:javafaker:1.0.2

import com.github.javafaker.Faker;

Faker faker = new Faker();

faker.name().fullName();


In [ ]:
import org.dflib.jjava.jupyter.kernel.display.mime.MIMEType;
import java.util.Set;
import java.util.HashSet;

kernel().getRenderer()
    .createRegistration(dev.jbang.jupyter.JBangHelper.JBangInfo.class)
    .preferring(MIMEType.TEXT_HTML)
    .register((info, ctx) -> {
        ctx.renderIfRequested(MIMEType.TEXT_HTML, () -> {
            StringBuilder html = new StringBuilder();
            
            // Create set of new dependencies for quick lookup
            Set<String> newDepsSet = new HashSet<>();
            if (info.getNewResolvedDependencies() != null) {
                newDepsSet.addAll(info.getNewResolvedDependencies());
            }
            
            // Summary section
            int totalRequested = (info.getDependencies() != null) ? info.getDependencies().size() : 0;
            int totalResolved = (info.getResolvedDependencies() != null) ? info.getResolvedDependencies().size() : 0;
            int totalNew = (info.getNewResolvedDependencies() != null) ? info.getNewResolvedDependencies().size() : 0;
            
            // Styles
            html.append("<style>\n");
            html.append(".jbang-compact { font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, Oxygen, Ubuntu, Cantarell, sans-serif; font-size: 12px; margin: 4px 0; }\n");
            html.append(".jbang-inline { display: inline-block; margin: 0; padding: 0; vertical-align: top; }\n");
            html.append(".jbang-inline summary { display: inline; cursor: pointer; user-select: none; padding: 2px 6px; border-radius: 3px; transition: background 0.15s; }\n");
            html.append(".jbang-inline summary:hover { background: #f1f3f5; }\n");
            html.append(".jbang-inline-content { margin-top: 6px; padding: 8px; background: #f8f9fa; border-radius: 4px; border-left: 2px solid #667eea; }\n");
            html.append(".jbang-inline-new .jbang-inline-content { border-left-color: #10b981; }\n");
            html.append(".jbang-inline-coord .jbang-inline-content { border-left-color: #764ba2; }\n");
            html.append(".jbang-list { list-style: none; padding: 0; margin: 0; font-size: 11px; }\n");
            html.append(".jbang-list-item { padding: 4px 6px; margin: 2px 0; background: white; border-radius: 2px; border-left: 2px solid #667eea; font-family: 'Monaco', 'Menlo', monospace; color: #2d3748; word-break: break-all; cursor: help; }\n");
            html.append(".jbang-list-item:hover { background: #f8f9fa; }\n");
            html.append(".jbang-list-item-new { border-left-color: #10b981; }\n");
            html.append(".jbang-list-item-old { opacity: 0.4; border-left-color: #adb5bd; }\n");
            html.append(".jbang-list-item-old:hover { opacity: 0.6; }\n");
            html.append(".jbang-list-item-coord { border-left-color: #764ba2; cursor: default; }\n");
            html.append("</style>\n");
            
            if (totalRequested > 0 || totalResolved > 0) {
                html.append("<div class='jbang-compact'>\n");
                html.append("  🚀 JBang: ");
            
                // Requested dependencies - always show, inline expandable if > 0
                if (info.getDependencies() != null && !info.getDependencies().isEmpty()) {
                    html.append("<details class='jbang-inline jbang-inline-coord'><summary title='Requested Maven dependencies (click to expand)'>📦 <strong>").append(totalRequested).append("</strong></summary>");
                    html.append("<div class='jbang-inline-content'><ul class='jbang-list'>");
                    for (String dep : info.getDependencies()) {
                        html.append("<li class='jbang-list-item jbang-list-item-coord'>").append(dep).append("</li>");
                    }
                    html.append("</ul></div></details>");
                } else {
                    html.append("<span title='Requested Maven dependencies'>📦 <strong>").append(totalRequested).append("</strong></span>");
                }
                
                html.append(" | ");
                
                // New dependencies - always show, inline expandable if > 0
                if (info.getNewResolvedDependencies() != null && !info.getNewResolvedDependencies().isEmpty()) {
                    html.append("<details class='jbang-inline jbang-inline-new'><summary title='Newly added to classpath (click to expand)'>✨ <strong>").append(totalNew).append("</strong></summary>");
                    html.append("<div class='jbang-inline-content'><ul class='jbang-list'>");
                    for (String dep : info.getNewResolvedDependencies()) {
                        String fileName = dep.substring(dep.lastIndexOf('/') + 1);
                        html.append("<li class='jbang-list-item jbang-list-item-new' title='").append(dep).append("'>").append(fileName).append("</li>");
                    }
                    html.append("</ul></div></details>");
                } else {
                    html.append("<span title='Newly added to classpath'>✨ <strong>").append(totalNew).append("</strong></span>");
                }
                
                html.append(" | ");
                
                // All resolved dependencies - always show, inline expandable if > 0
                if (info.getResolvedDependencies() != null && !info.getResolvedDependencies().isEmpty()) {
                    html.append("<details class='jbang-inline'><summary title='All resolved JARs (click to expand, grayed = cached)'>📚 <strong>").append(totalResolved).append("</strong></summary>");
                    html.append("<div class='jbang-inline-content'><ul class='jbang-list'>");
                    for (String dep : info.getResolvedDependencies()) {
                        String fileName = dep.substring(dep.lastIndexOf('/') + 1);
                        boolean isNew = newDepsSet.contains(dep);
                        String itemClass = isNew ? "jbang-list-item jbang-list-item-new" : "jbang-list-item jbang-list-item-old";
                        html.append("<li class='").append(itemClass).append("' title='").append(dep).append("'>").append(fileName).append("</li>");
                    }
                    html.append("</ul></div></details>");
                } else {
                    html.append("<span title='All resolved JARs'>📚 <strong>").append(totalResolved).append("</strong></span>");
                }
                
                html.append("</div>\n");
            } else {
                html.append("<div class='jbang-compact'>🚀 JBang: <em>No dependencies</em></div>\n");
            }
            
            return html.toString();
        });
    });

In [ ]:
System.getenv();


In [ ]:
var id = display("Markdown *test* **snippet**:\n* bullet 1\n* bullet 2", "text/markdown")

## Built-in displays

Images gets rendered automatically.

In [ ]:
//DEPS org.knowm.xchart:xchart:3.8.8
import org.knowm.xchart.*;
import java.awt.Color;

In [ ]:
PieChart chart = new PieChartBuilder().width(800).height(600).title("Pie!").build();
chart.getStyler().setChartBackgroundColor(Color.WHITE);
chart.getStyler().setPlotBackgroundColor(Color.WHITE);
chart.getStyler().setAntiAlias(true);
 
// Customize Chart
Color[] sliceColors = new Color[] {
    new Color(0, 191, 255), // vibrant sky blue
    new Color(255, 223, 0),   // bright yellow
    new Color(255, 236, 139),  // shaded yellow
};
chart.getStyler().setSeriesColors(sliceColors);
 
// Series
chart.addSeries("Sky", 80);
chart.addSeries("Sunny side of Pyramid", 15);
chart.addSeries("Shady side of Pyramid", 10);

BitmapEncoder.getBufferedImage(chart);

# Vega rendering

Using smile-plot/vega that can generate Vega-Lite json you can return the json string with the right mime type to have it rendered.

In [ ]:
//DEPS com.github.haifengl:smile-plot:4.3.0

import java.lang.Math.*;
//import smile.*;
//import smile.json.*;
import smile.plot.vega.*;


In [ ]:
var bar = new View("Simple Bar Plot")
        .description("A simple bar chart with embedded data.")
        .widthStep(30);
bar.background(null);
bar.data().values("""
        [
          {"a": "A", "b": 28}, {"a": "B", "b": 55}, {"a": "C", "b": 43},
          {"a": "D", "b": 91}, {"a": "E", "b": 81}, {"a": "F", "b": 53},
          {"a": "G", "b": 19}, {"a": "H", "b": 87}, {"a": "I", "b": 52}
        ]""");

bar.mark("bar");
Field x = bar.encode("x", "a").type("ordinal");
x.axis().labelAngle(23);
bar.encode("y", "b").type("quantitative");
display(bar.toPrettyString(), "application/vnd.vegalite.v5+json");